# Preparación del Escenario

In [19]:
import pandas as pd
import numpy as np

# Datos crudos extraídos de la base de datos de la tienda
data_ecommerce = {
    'id_pedido': ['PED-001', 'PED-002', 'PED-003', 'PED-004', 'PED-005'],
    'producto': ['Laptop', 'Ratón', 'Escritorio', 'Teclado', 'Monitor'],
    'categoria': ['Tecnología', 'Tecnología', 'Muebles', 'Tecnología', 'Tecnología'],
    'precio_base': [1000.0, 25.0, 300.0, 45.0, 250.0],
    'cantidad': [1, 3, 1, 10, 2],
    'tipo_cliente': ['Normal', 'Normal', 'VIP', 'VIP', 'Normal']
}

df_ventas = pd.DataFrame(data_ecommerce)
print("--- DATOS CRUDOS DE VENTAS ---")
print(df_ventas)

--- DATOS CRUDOS DE VENTAS ---
  id_pedido    producto   categoria  precio_base  cantidad tipo_cliente
0   PED-001      Laptop  Tecnología       1000.0         1       Normal
1   PED-002       Ratón  Tecnología         25.0         3       Normal
2   PED-003  Escritorio     Muebles        300.0         1          VIP
3   PED-004     Teclado  Tecnología         45.0        10          VIP
4   PED-005     Monitor  Tecnología        250.0         2       Normal


# Ejemplo 1: Operaciones Matemáticas (Vectorización)

Contexto Real: Tienes el precio de 1 unidad y cuántas unidades se llevaron. Necesitamos la columna subtotal. En Excel harías C2 * D2 y arrastrarías hacia abajo. En Pandas, hacemos esto:

In [20]:
# Creamos la nueva columna 'subtotal' multiplicando dos columnas existentes
df_calculando = df_ventas.assign(
    subtotal= lambda  x:x['precio_base'] * x['cantidad']
)

print('--- Ventas con Subtotal---')
print((df_calculando[['producto','precio_base','cantidad','subtotal']]))

--- Ventas con Subtotal---
     producto  precio_base  cantidad  subtotal
0      Laptop       1000.0         1    1000.0
1       Ratón         25.0         3      75.0
2  Escritorio        300.0         1     300.0
3     Teclado         45.0        10     450.0
4     Monitor        250.0         2     500.0


Explicación:

    .assign(subtotal = ...): Le decimos a Pandas: "Crea una columna que se llame subtotal (nota que no lleva comillas el nombre de la variable aquí)".

    lambda x:: x es nuestro DataFrame temporal.

    x['precio_base'] * x['cantidad']: Multiplica fila por fila al instante. No hay bucles, lo hace usando los motores optimizados de Python en fracciones de segundo.

# Ejemplo 2: Lógica Condicional de Negocio (np.where)

Contexto Real: Las reglas de la empresa dicen:
"Si el producto es de la categoría 'Muebles', el envío cuesta 50 (porque son pesados). Para cualquier otra cosa, el envío cuesta 10"

In [21]:
#Usamos np.where, aplicamos la regla de negocio para calcular el envío
df_envios = df_ventas.assign(
    costo_envio=lambda x:np.where(x['categoria']=='Muebles',50.0,10.0)
)

print("---- Costos de Envío Calculados---")
print(df_envios[['producto', 'categoria', 'costo_envio']])

---- Costos de Envío Calculados---
     producto   categoria  costo_envio
0      Laptop  Tecnología         10.0
1       Ratón  Tecnología         10.0
2  Escritorio     Muebles         50.0
3     Teclado  Tecnología         10.0
4     Monitor  Tecnología         10.0


Explicación:

    np.where( CONDICIÓN, VALOR_SI_CUMPLE, VALOR_SI_NO_CUMPLE ).

    x['categoria'] == 'Muebles': Esta es la pregunta. Evalúa cada fila.

    50.0: Es lo que pone si la respuesta a la pregunta es "Sí".

    10.0: Es lo que pone si la respuesta es "No".

# Ejemplo 3: Condiciones Múltiples (El VIP)

Contexto Real: Ahora queremos crear una etiqueta de prioridad para el almacén.
La regla es: "Si el cliente es 'VIP' O la cantidad comprada es mayor a 5 unidades, márcalo como 'Alta Prioridad'. Si no, 'Normal'".

In [22]:
# Usamos el operador | (OR) para unir dos condiciones dentro del np.where

df_prioridad = df_ventas.assign(
    prioridad_almacen = lambda x: np.where(
        (x['tipo_cliente'] == 'VIP')| (x['cantidad'] > 5),
        'Alta Prioridad',
        'Normal'
    )
)

print("---- Etiquetas de almacén----")
print(df_prioridad[['id_pedido', 'tipo_cliente', 'cantidad', 'prioridad_almacen']])

---- Etiquetas de almacén----
  id_pedido tipo_cliente  cantidad prioridad_almacen
0   PED-001       Normal         1            Normal
1   PED-002       Normal         3            Normal
2   PED-003          VIP         1    Alta Prioridad
3   PED-004          VIP        10    Alta Prioridad
4   PED-005       Normal         2            Normal


# Ejemplo 4: El "Pipeline" Maestro (Estilo Effective Pandas) 🏆

Contexto Real: En el trabajo diario, no creas una variable nueva (df_calculado, df_envios) por cada columna. Haces todo en una sola tubería fluida (Pipeline).
Aquí calcularemos el subtotal, el envío y un Total Final, todo encadenado.

In [24]:
# El pipeline definitivo: Todo en un solo flujo limpio
df_reporte_final=(df_ventas
    #1. Calculamos el subtotal
    .assign(subtotal=lambda x:x['precio_base'] * x['cantidad'])

    # 2. Calculamos el envío (Muebles 50, otros 10)
    .assign(costo_envio = lambda x:np.where(x['categoria']=='Muebles', 50.0, 10.0))

    #3. Calculamos el total usando las columnas que acabamos de crear arriba
    .assign(total_pagar = lambda x: x['subtotal'] + x['costo_envio'])
)

print("--- Reporte Financiero Final----")
print(df_reporte_final[['id_pedido', 'subtotal', 'costo_envio', 'total_pagar']])

--- Reporte Financiero Final----
  id_pedido  subtotal  costo_envio  total_pagar
0   PED-001    1000.0         10.0       1010.0
1   PED-002      75.0         10.0         85.0
2   PED-003     300.0         50.0        350.0
3   PED-004     450.0         10.0        460.0
4   PED-005     500.0         10.0        510.0


Explicación:

    Fíjate en el paso 3 (total_pagar). Estamos sumando x['subtotal'] + x['costo_envio'].

    Esas dos columnas no existían en el DataFrame original.

    ¿Por qué funciona? Porque lambda x: toma el DataFrame exactamente como quedó en la línea anterior. Como en la línea 1 y 2 ya creamos esas columnas, la línea 3 ya puede verlas y usarlas.

    Este es el verdadero superpoder de .assign(): construir datos paso a paso sin ensuciar la memoria con variables intermedias.